In [1]:
%matplotlib inline
import theano
floatX = theano.config.floatX
import pymc3 as pm
import theano.tensor as T
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split

from sklearn.datasets import fetch_mldata
import scipy.io as sio


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu0 is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [2]:
# mnist = fetch_mldata('MNIST original', data_home='.')
### to download MNIST data, uncomment the line above and run ###
mnist = sio.loadmat('mldata/mnist-original.mat')
data = mnist['data'].T
label = mnist['label'].T

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=1.0/7)
X_train = X_train.astype(floatX)
X_test = X_test.astype(floatX)
X_train /= 255
X_test /= 255

In [8]:
def construct_nn(ann_input, ann_output):
    n_hidden = 5
    n_classes = 10

    # Initialize random weights between each layer
    init_1 = np.random.randn(data.shape[1], n_hidden).astype(floatX)
    init_2 = np.random.randn(n_hidden, n_classes).astype(floatX)
    init_out = np.random.randn(n_classes).astype(floatX)

    with pm.Model() as neural_network:
        # Weights from input to hidden layer
        weights_in_1 = pm.Normal('w_in_1', 0, sd=1,
                                 shape=(data.shape[1], n_hidden),
                                 testval=init_1)

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal('w_1_2', 0, sd=1,
                                shape=(n_hidden, n_classes),
                                testval=init_2)

        # Weights from hidden layer to output
        weights_2_out = pm.Normal('w_2_out', 0, sd=1,
                                  shape=(n_classes,),
                                  testval=init_out)

        # Build neural-network using tanh activation function
        act_1 = pm.math.tanh(pm.math.dot(ann_input,
                                         weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1,
                                         weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2,
                                              weights_2_out))

        # Categorical classification (10 classes)
        out = pm.Categorical('out',
                           act_out,
                           observed=ann_output,
                           total_size=Y_train.shape[0] # IMPORTANT for minibatches
                          )
    return neural_network


In [9]:
ann_input = theano.shared(X_train)
ann_output = theano.shared(Y_train)
neural_network = construct_nn(ann_input, ann_output)

In [10]:
with neural_network:
    step = pm.Metropolis() # Should be NUTS, but veeeeeeeery slow
    trace = pm.sample(10000, init='advi', step=step, chains=1)

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [w_2_out]
>Metropolis: [w_1_2]
>Metropolis: [w_in_1]
  1%|          | 69/10500 [01:00<2:32:12,  1.14it/s]
